In [37]:
import pandas as pd
import ast
import os
import math

os.chdir("/Users/zaza/Valorant Match Predictor")
df = pd.read_csv("data/processed/R2_processed/R2_unified_dataset.csv")

# Parse string representations of lists
df['all_player_ratings'] = df['all_player_ratings'].apply(ast.literal_eval)
df['all_player_teams'] = df['all_player_teams'].apply(ast.literal_eval)
df['all_players'] = df['all_players'].apply(ast.literal_eval)

In [38]:
def remove_double_back_and_forth(L):
    n = len(L)
    if n % 2 == 0:
        half = n // 2
        # if the first half exactly equals the second half, keep only the first
        if L[:half] == L[half:]:
            return L[:half]
    return L

# Apply to all columns
for col in ['all_players', 'all_player_teams', 'all_player_ratings']:
    df[col] = df[col].apply(remove_double_back_and_forth)

# Sanity check
lengths = df[['all_players','all_player_teams','all_player_ratings']].applymap(len).head()
print(lengths)

# Define your 43 teams → list of valid codes (new + legacy)
team_abbreviations = {
    "Sentinels":          ["SEN"],
    "100 Thieves":        ["100T"],
    "Cloud9":             ["C9"],
    "NRG":                ["NRG"],
    "Evil Geniuses":      ["EG"],
    "G2 Esports":         ["G2"],
    "LOUD":               ["LOUD"],
    "MIBR":               ["MIBR"],
    "FURIA":              ["FUR"],
    "KRÜ Esports":        ["KRÜ"],
    "LEVIATÁN":           ["LEV"],
    "FNATIC":             ["FNC"],
    "Team Liquid":        ["TL"],
    "Team Heretics":      ["TH"],
    "BBL Esports":        ["BBL"],
    "FUT Esports":        ["FUT"],
    "Karmine Corp":       ["KC"],
    "Team Vitality":      ["VIT"],
    "Natus Vincere":      ["NAVI"],
    "Gentle Mates":       ["M8"],
    "Apeks":              ["APK"],
    "Paper Rex":          ["PRX"],
    "DRX":                ["DRX"],
    "Gen.G":              ["GEN"],
    "T1":                 ["T1"],
    "Rex Regum Qeon":     ["RRQ"],
    "TALON":              ["TLN"],
    "Team Secret":        ["TS"],
    "ZETA DIVISION":      ["ZETA","FL"],
    "DetonatioN FocusMe": ["DFM","CR"],
    "Global Esports":     ["GE"],
    "EDward Gaming":      ["EDG"],
    "Bilibili Gaming":    ["BLG"],
    "FunPlus Phoenix":    ["FPX"],
    "Dragon Ranger Gaming":["DRG"],
    "Wolves Esports":     ["WOL"],
    "Trace Esports":      ["TE"],
    "Titan Esports Club": ["TEC"],
    "Nova Esports":       ["NOVA"],
    "All Gamers":         ["AG"],
    "TYLOO":              ["TYL"],
    "JDG Esports":        ["JDG"],
    "Rare Atom":          ["RA"],
}

# Build set of all valid codes
valid_codes = {c for codes in team_abbreviations.values() for c in codes}

# Flag rows missing any valid code
df['has_valid_team'] = df['all_player_teams'].apply(lambda teams: any(c in valid_codes for c in teams))

# Print invalid matches (no valid code)
print("Matches with no recognized team code:")
for _, row in df[~df['has_valid_team']].iterrows():
    print(f"{row['team_name']} — {row['match_url']}")

   all_players  all_player_teams  all_player_ratings
0           10                10                  10
1           10                10                  10
2           10                10                  10
3           10                10                  10
4           10                10                  10
Matches with no recognized team code:


/var/folders/j1/0ttm8wnn2hx0rydcd55zy81r0000gn/T/ipykernel_2952/1902155580.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  lengths = df[['all_players','all_player_teams','all_player_ratings']].applymap(len).head()


In [39]:
def split_mean_ratings(row):
    teams = row['all_player_teams']
    ratings = row['all_player_ratings']
    
    codes = team_abbreviations[row['team_name']]
    
    my_rates = [r for t,r in zip(teams, ratings) if t in codes]
    enemy_rates = [r for t,r in zip(teams, ratings) if t not in codes]
    
    average_my_rate = sum(my_rates) / len(my_rates) if my_rates else math.nan
    average_enemy_rate = sum(enemy_rates) / len(enemy_rates) if enemy_rates else math.nan
    
    return pd.Series({
        'mean_my_rating':    average_my_rate,
        'mean_opp_rating':   average_enemy_rate
    })

df[['mean_my_rating','mean_opp_rating']] = df.apply(split_mean_ratings, axis=1)

In [42]:
# Alternative approach - more explicit
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
df = df.sort_values(['team_id', 'date']).reset_index(drop=True)

# Initialize columns
df['rolling_30d_my'] = pd.NA
df['rolling_30d_opp'] = pd.NA

# Calculate rolling averages for each team
for team_id in df['team_id'].unique():
    team_mask = df['team_id'] == team_id
    team_data = df[team_mask].copy()
    
    # Calculate rolling means
    team_data = team_data.set_index('date').sort_index()
    rolling_my = team_data['mean_my_rating'].rolling('30D', min_periods=2).mean()
    rolling_opp = team_data['mean_opp_rating'].rolling('30D', min_periods=2).mean()
    
    # Assign back to main dataframe
    df.loc[team_mask, 'rolling_30d_my'] = rolling_my.values
    df.loc[team_mask, 'rolling_30d_opp'] = rolling_opp.values

# Convert to float
df['rolling_30d_my'] = df['rolling_30d_my'].astype('float64')
df['rolling_30d_opp'] = df['rolling_30d_opp'].astype('float64')

In [46]:
# Show the first 10 rows with the new columns
print("First 10 rows with rolling averages:")
print(df[['team_name', 'date', 'mean_my_rating', 'rolling_30d_my', 
         'mean_opp_rating', 'rolling_30d_opp']].head(10))

# Show overall averages
print("\nOverall averages:")
print(f"Mean my rating: {df['mean_my_rating'].mean():.2f}")
print(f"Mean opponent rating: {df['mean_opp_rating'].mean():.2f}")
print(f"Mean 30-day rolling my rating: {df['rolling_30d_my'].mean():.2f}")
print(f"Mean 30-day rolling opponent rating: {df['rolling_30d_opp'].mean():.2f}")

# Show averages by team (top 10 teams)
print("\nAverages by team (top 10):")
team_avgs = df.groupby('team_name').agg({
    'mean_my_rating': 'mean',
    'mean_opp_rating': 'mean',
    'rolling_30d_my': 'mean',
    'rolling_30d_opp': 'mean'
}).round(2)
print(team_avgs.head(10))

# Show descriptive statistics
print("\nDescriptive statistics:")
print(df[['mean_my_rating', 'mean_opp_rating', 'rolling_30d_my', 'rolling_30d_opp']].describe())

First 10 rows with rolling averages:
   team_name       date  mean_my_rating  rolling_30d_my  mean_opp_rating  \
0  Sentinels 2024-05-03           1.146             NaN            0.828   
1  Sentinels 2024-06-22           1.200             NaN            0.820   
2  Sentinels 2024-06-29           1.104        1.152000            0.900   
3  Sentinels 2024-07-01           1.144        1.149333            0.826   
4  Sentinels 2024-07-14           0.826        1.068500            1.190   
5  Sentinels 2024-07-17           0.912        1.037200            1.082   
6  Sentinels 2024-08-01           0.856        0.864667            1.132   
7  Sentinels 2024-08-08           1.210        0.951000            0.794   
8  Sentinels 2024-08-10           1.210        1.002800            0.794   
9  Sentinels 2024-08-14           1.108        1.059200            0.920   

   rolling_30d_opp  
0              NaN  
1              NaN  
2         0.860000  
3         0.848667  
4         0.934000  


In [ ]:
# Check for duplicates
print("Checking for duplicate matches:")
duplicate_check = df[df['team_name'] == 'Sentinels'][['date', 'mean_my_rating', 'mean_opp_rating', 'match_url']].head(10)
print(duplicate_check)

# Check the rating distribution
print("\nRating distribution for Sentinels:")
sen_data = df[df['team_name'] == 'Sentinels']
print(f"Min rating: {sen_data['mean_my_rating'].min():.3f}")
print(f"Max rating: {sen_data['mean_my_rating'].max():.3f}")
print(f"Std deviation: {sen_data['mean_my_rating'].std():.3f}")

# Check what the ratings represent
print("\nSample of individual player ratings from one match:")
sample_match = df[df['team_name'] == 'Sentinels'].iloc[7]  # the 1.210 match
print(f"All player ratings: {sample_match['all_player_ratings']}")
print(f"All player teams: {sample_match['all_player_teams']}")

Checking for duplicate matches:
        date  mean_my_rating  mean_opp_rating  \
0 2024-05-03           1.146            0.828   
1 2024-06-22           1.200            0.820   
2 2024-06-29           1.104            0.900   
3 2024-07-01           1.144            0.826   
4 2024-07-14           0.826            1.190   
5 2024-07-17           0.912            1.082   
6 2024-08-01           0.856            1.132   
7 2024-08-08           1.210            0.794   
8 2024-08-10           1.210            0.794   
9 2024-08-14           1.108            0.920   

                                           match_url  
0  https://www.vlr.gg/314650/sentinels-vs-furia-c...  
1  https://www.vlr.gg/353178/sentinels-vs-nrg-esp...  
2  https://www.vlr.gg/353183/sentinels-vs-kr-espo...  
3  https://www.vlr.gg/353187/sentinels-vs-cloud9-...  
4  https://www.vlr.gg/353201/sentinels-vs-g2-espo...  
5  https://www.vlr.gg/371267/sentinels-vs-100-thi...  
6  https://www.vlr.gg/378662/gen-g-vs-senti